This notebook simply goes to a number of positions in workd space and reads out the joint configugarion of the robot at the position.
These data is used in `scene_utils` to workaround the limitation of a missing exposed inverse kinematics solver as well as the missing `beam_to_cart_pos` method.
The data is collected in a grid pattern in the workspace of the robot.

In [15]:
from pprint import pprint

import numpy as np
import glfw

from alr_sim.sims.SimFactory import SimRepository
from alr_sim.core import Scene

from alr_sim_tools.scene_utils import reset_scene

In [16]:
FACTORY_SRTING = "mj_beta"
DT=0.0002
RENDER_MODE = Scene.RenderMode.HUMAN
CAMERA_POSITION = (0.5, 0.0, 1.0)
CAMERA_QUAT = (-0.70710678, 0, 0, 0.70710678)

MOVEMENT_DURATION = 2
WAIT_DURATIONS_BETWEEN_MOVEMENTS = 0.5
TARGET_QUAT = (0, 1, 0, 0)

MIN_X = 0.0
MAX_X = 0.0
DELTA_X = 0.1
MIN_Y = 0.5
MAX_Y = 0.5
DELTA_Y = 0.1
MIN_Z = 0.1
MAX_Z = 0.6
DELTA_Z = 0.05

JOINT_VALUE_DECIMALS = 5

In [17]:
positions = np.meshgrid(
    np.arange(MIN_X, MAX_X + DELTA_X, DELTA_X),
    np.arange(MIN_Y, MAX_Y + DELTA_Y, DELTA_Y),
    np.arange(MIN_Z, MAX_Z + DELTA_Z, DELTA_Z),
)
positions = np.array(positions).T.reshape(-1, 3)

positions


array([[0.  , 0.5 , 0.1 ],
       [0.  , 0.5 , 0.15],
       [0.  , 0.5 , 0.2 ],
       [0.  , 0.5 , 0.25],
       [0.  , 0.5 , 0.3 ],
       [0.  , 0.5 , 0.35],
       [0.  , 0.5 , 0.4 ],
       [0.  , 0.5 , 0.45],
       [0.  , 0.5 , 0.5 ],
       [0.  , 0.5 , 0.55],
       [0.  , 0.5 , 0.6 ]])

In [18]:
sim_factory = SimRepository.get_factory(FACTORY_SRTING)
scene = sim_factory.create_scene(object_list=[], dt=DT, render=RENDER_MODE)
agent = sim_factory.create_robot(scene, dt=DT)

scene.start()
scene.start_logging()

In [19]:
world_2_joint_coords = {}

for pos in positions:
    pos = tuple(pos.tolist())
    print(f"Going to {pos}") 
    agent.gotoCartPositionAndQuat(pos, TARGET_QUAT, MOVEMENT_DURATION)
    agent.wait(WAIT_DURATIONS_BETWEEN_MOVEMENTS)
    world_2_joint_coords[pos] = tuple(agent.current_j_pos) 


Going to (0.0, 0.5, 0.1)
Going to (0.0, 0.5, 0.15000000000000002)
Going to (0.0, 0.5, 0.20000000000000004)
Going to (0.0, 0.5, 0.25000000000000006)
Going to (0.0, 0.5, 0.30000000000000004)
Going to (0.0, 0.5, 0.3500000000000001)
Going to (0.0, 0.5, 0.40000000000000013)
Going to (0.0, 0.5, 0.45000000000000007)
Going to (0.0, 0.5, 0.5000000000000001)
Going to (0.0, 0.5, 0.5500000000000002)
Going to (0.0, 0.5, 0.6000000000000002)


In [20]:
scene.stop_logging()
reset_scene(FACTORY_SRTING, scene, agent)
glfw.terminate()

In [21]:
world_2_joint_coords_ = {}

for pos, j_coords in world_2_joint_coords.items():
    world_2_joint_coords_[(tuple(pos), tuple(TARGET_QUAT))] = tuple(
        round(j, JOINT_VALUE_DECIMALS) for j in j_coords
    )

pprint(world_2_joint_coords_, width=160)

{((0.0, 0.5, 0.1), (0, 1, 0, 0)): (1.08803, 0.30093, 0.4463, -2.38944, -0.2721, 2.64706, 2.55064),
 ((0.0, 0.5, 0.15000000000000002), (0, 1, 0, 0)): (0.97897, 0.18506, 0.57862, -2.39467, -0.18042, 2.54396, 2.48678),
 ((0.0, 0.5, 0.20000000000000004), (0, 1, 0, 0)): (0.87854, 0.06244, 0.69158, -2.37661, -0.06162, 2.42568, 2.4012),
 ((0.0, 0.5, 0.25000000000000006), (0, 1, 0, 0)): (0.80679, -0.05558, 0.76152, -2.33542, 0.05036, 2.29661, 2.31926),
 ((0.0, 0.5, 0.30000000000000004), (0, 1, 0, 0)): (0.75482, -0.15696, 0.79618, -2.27291, 0.13406, 2.16064, 2.2552),
 ((0.0, 0.5, 0.3500000000000001), (0, 1, 0, 0)): (0.70971, -0.23439, 0.81244, -2.19119, 0.18799, 2.02354, 2.21079),
 ((0.0, 0.5, 0.40000000000000013), (0, 1, 0, 0)): (0.66535, -0.28557, 0.82181, -2.09133, 0.21849, 1.88887, 2.18294),
 ((0.0, 0.5, 0.45000000000000007), (0, 1, 0, 0)): (0.62226, -0.30905, 0.82974, -1.97321, 0.2301, 1.75794, 2.17018),
 ((0.0, 0.5, 0.5000000000000001), (0, 1, 0, 0)): (0.5863, -0.30165, 0.83854, -1.83541,